# Trabalho 2 de Paradigmas da Programação

## Bibliotecas

In [19]:
import pandas as pd
import sparql_dataframe

## Sparql query

In [2]:
endpoint = "http://dbpedia.org/sparql"

In [3]:
query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select distinct ?nome ?paisNascimento ?dataNascimentoFormatted ?clubeNome ?posicaoLabel ?altura ?numeroCamisa ?allGols ?liga where {
  
  # Pegando as instâncias de SoccerPlayer e o nome do jogador.
  { ?sub rdf:type dbo:SoccerPlayer . }
  union
  { ?sub a dbo:SoccerPlayer . }
  ?sub rdfs:label ?nome .
  FILTER (LANG(?nome) = 'en') .
  
  # Pegando o país de nascimento do jogador.
  ?sub dbo:birthPlace ?localNascimento .
  OPTIONAL { 
    ?localNascimento dbo:country ?paisNascimentoURI . 
  	?paisNascimentoURI rdfs:label ?paisNascimento .
   	FILTER (LANG(?paisNascimento) = 'en') .
  }
  
  # Pegando a data de nascimento do jogador.
  ?sub dbp:birthDate ?dataNascimento .
  
  # Pegando o clube em que o jogador está jogando.
  ?sub dbp:currentclub ?clubeAtual .
  ?clubeAtual rdfs:label ?clubeNome .
  FILTER (LANG(?clubeNome) = 'en') .
  
  # Pegando a posição em que o jogador joga.
  ?sub dbp:position ?posicao .
  ?posicao rdfs:label ?posicaoLabel .
  FILTER (LANG(?posicaoLabel) = 'en') .
  
  # Pegando a altura do jogador.
  ?sub dbo:height ?altura .
  FILTER (datatype(?altura) = xsd:double) .
  
  # Pegando o número da camisa do jogador.
  ?sub dbp:clubnumber ?numeroCamisa .
  FILTER (datatype(?numeroCamisa) = xsd:integer) .
  
  # Pegando a lista de gols do jogador, onde apenas o máximo dessa lista representa a quantidade real de gols dele.
  ?sub dbp:goals ?allGols .
  FILTER (datatype(?allGols) = xsd:integer) .
  
  # Pegando a liga em que o jogador joga.
  ?clubeAtual dbo:league ?ligaURI .
  ?ligaURI rdfs:label ?liga .
  FILTER (LANG(?liga) = 'en') .
  
  
  # Filtrando variáveis que devem ser URIs.
  FILTER (isURI(?clubeAtual)) .
  FILTER (isURI(?paisNascimentoURI)) .
  
  # Transformando as datas de nascimento do formato dateTime para date.
  BIND (
    IF (datatype(?dataNascimento) = xsd:dateTime, STRDT(STR(?dataNascimento), xsd:date), ?dataNascimento)
    AS ?dataNascimentoFormatted
  )
}"""

In [35]:
df = sparql_dataframe.get(endpoint, query)
df.head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
0,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,0,Calcutta Football League
1,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,1,Calcutta Football League
2,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,2,Calcutta Football League
3,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,7,Calcutta Football League
4,Nub Tola,Cambodia,1995-10-01,Nagaworld FC,Defender (association football),1.75,13,3,Cambodian Premier League


In [36]:
df.shape

(10000, 9)

Iremos filtrar o dataframe para tirar as repetições de jogadores

Nesta parte foi utilizada a funcionalidade **```filter```**

In [37]:
jogadores_list = df.to_dict(orient='records')

jogadores_filtrados = list(filter(lambda j: j['allGols'] == max(
    jogador['allGols'] for jogador in jogadores_list if jogador['nome'] == j['nome']), jogadores_list))

In [38]:
new_df = pd.DataFrame(jogadores_filtrados)
new_df.head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
0,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,7,Calcutta Football League
1,Nub Tola,Cambodia,1995-10-01,Nagaworld FC,Defender (association football),1.75,13,3,Cambodian Premier League
2,Nen Sothearoth,Cambodia,1995-12-24,Nagaworld FC,Defender (association football),1.73,7,1,Cambodian Premier League
3,Chhom Pisa,Cambodia,1995-03-03,Phnom Penh Crown FC,Centre back,1.70,39,0,Cambodian Premier League
4,Sor Rotana,Cambodia,2002-10-09,Prey Veng FC,Defender (association football),1.84,4,19,Cambodian League 2


In [39]:
new_df.shape

(3019, 9)

## Regras

### Regra 1

Acha todos os jogadores que são conteporâneos, ou seja, nasceram no mesmo ano

Nesta regra foi utilizada a funcionalidade **```map```**

In [40]:
def jogadores_contemporaneos(df):
    return [(j1, j2) for j1, ano1 in zip(df['nome'], list(map(lambda date: int(date[:4]) if '-' in date else None, df["dataNascimentoFormatted"])))
                     for j2, ano2 in zip(df['nome'], list(map(lambda date: int(date[:4]) if '-' in date else None, df["dataNascimentoFormatted"])))
                     if j1 < j2 and ano1 == ano2]

In [41]:
jogadores_contemporaneos(new_df)

[('Tirthankar Sarkar', 'Vinícius (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Zé Roberto (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Zé Rafael'),
 ('Tirthankar Sarkar', 'Zé Rafael'),
 ('Tirthankar Sarkar', 'Václav Klán'),
 ('Tirthankar Sarkar', 'Ângelo Meneses'),
 ('Tirthankar Sarkar', 'Élder Santana'),
 ('Tirthankar Sarkar', 'Wilson Kenidy'),
 ('Tirthankar Sarkar', 'Tomás Spinelli'),
 ('Tirthankar Sarkar', 'Welinton Júnior'),
 ('Tirthankar Sarkar', 'Wendell (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Tomás Paschetta'),
 ('Tirthankar Sarkar', 'Yohan Boli'),
 ('Tirthankar Sarkar', 'Vitinho (footballer, born October 1993)'),
 ('Tirthankar Sarkar', 'Vitinho (footballer, born October 1993)'),
 ('Tirthankar Sarkar', 'Víctor Ruiz (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Vadim Balbukh'),
 ('Tirthankar Sarkar', 'Yegor Generalov'),
 ('Nub Tola', 'Ricardo Luz'),
 ('Nub Tola', 'Pedro 

### Regra 2

Acha todos os jogadores parceiros, ou seja, que jogam no mesmo clube

In [42]:
def jogadores_parceiros(df):
    return [(j1, j2) for j1, clube1 in zip(df['nome'], df['clubeNome'])
                     for j2, clube2 in zip(df['nome'], df['clubeNome'])
                     if j1 < j2 and clube1 == clube2]

In [44]:
jogadores_parceiros(new_df)

[('Nub Tola', 'Sou Yaty'),
 ('Nen Sothearoth', 'Nub Tola'),
 ('Nen Sothearoth', 'Sou Yaty'),
 ('Chhom Pisa', 'Valeriy Hryshyn'),
 ('Chhom Pisa', 'Lim Pisoth'),
 ('Chhom Pisa', 'In Sodavid'),
 ('Seon Power', 'Sor Rotana'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Víctor Blasco'),
 ('Sa Ty', 'Wut Tola'),
 ('Sa Ty', 'Thierry Bin'),
 ('Reung Bunheing', 'Sa Ty'),
 ('Reung Bunheing', 'Wut Tola'),
 ('Reung Bunheing', 'Thierry Bin'),
 ('Paulo Victor (footballer, born 1994)', 'Sa Ty'),
 ('Paulo Victor (footballer, born 1994)', 'Reung Bunheing'),
 ('Paulo Victor (footballer, born 1994)', 'Wut Tola'),
 ('Paulo Victor (footballer, born 1994)', 'Thierry Bin'),
 ('Lee Jae-gun', 'Sa Ty'),
 ('Lee Jae-gun', 'Reung Bunheing'),
 ('Lee Jae-gun', 'Paulo Victor (footballer, born 1994)'),
 ('Lee Jae-gun', 'Wut Tola'),
 ('Lee Jae-gun', 'Leng Nora'),
 ('Lee Jae-gun', 'Marcos Vinícius (footballer, born 1994)'),
 ('Lee Jae-gun', 'Thierry Bin'),
 ('Lee Jae-gun', 'Pak Song-chol (footbal

### Regra 3

Acha todos os jogadores que são super parceiros, ou seja, jogam no mesmo clube, nasceram no mesmo pais e jogam em posições diferentes

In [46]:
def jogadores_super_parceiros(df):
    return [(j1, j2, pais) for j1, clube1, pais, pos1 in zip(df['nome'], df['clubeNome'], df['paisNascimento'], df['posicaoLabel'])
                            for j2, clube2, pais2, pos2 in zip(df['nome'], df['clubeNome'], df['paisNascimento'], df['posicaoLabel'])
                            if j1 < j2 and clube1 == clube2 and pais == pais2 and pos1 != pos2]

In [48]:
jogadores_super_parceiros(new_df)

[('Nub Tola', 'Sou Yaty', 'Cambodia'),
 ('Nen Sothearoth', 'Sou Yaty', 'Cambodia'),
 ('Chhom Pisa', 'Lim Pisoth', 'Cambodia'),
 ('Chhom Pisa', 'In Sodavid', 'Cambodia'),
 ('Keo Soksela', 'Sa Ty', 'Cambodia'),
 ('Keo Soksela', 'Reung Bunheing', 'Cambodia'),
 ('Keo Soksela', 'Wut Tola', 'Cambodia'),
 ('Keo Soksela', 'Leng Nora', 'Cambodia'),
 ('In Sodavid', 'Lim Pisoth', 'Cambodia'),
 ('Leng Nora', 'Sa Ty', 'Cambodia'),
 ('Leng Nora', 'Reung Bunheing', 'Cambodia'),
 ('Leng Nora', 'Wut Tola', 'Cambodia'),
 ('Chrerng Polroth', 'Sa Ty', 'Cambodia'),
 ('Chrerng Polroth', 'Reung Bunheing', 'Cambodia'),
 ('Chrerng Polroth', 'Keo Soksela', 'Cambodia'),
 ('Chrerng Polroth', 'Wut Tola', 'Cambodia'),
 ('Chrerng Polroth', 'Leng Nora', 'Cambodia'),
 ('Marcos Vinícius (footballer, born 1994)',
  'Paulo Victor (footballer, born 1994)',
  'Brazil'),
 ('Hugo (footballer, born 1997)', 'Luan Dias', 'Brazil'),
 ('Hugo (footballer, born 1997)', 'Matheus Santos', 'Brazil'),
 ('Hugo (footballer, born 1997)', 

### Regra 4

Acha todos os centro avantes goleadores, ou seja, jogadores camisa 9, na posição de atacante que tenham feito mais de 50 gols

In [ ]:
def centro_avante_goleador(df):
    return df[(df['posicaoLabel'] == "Forward (association football)") &
              (df['numeroCamisa'] == 9) &
              (df['allGols'] > 50)]

In [50]:
centro_avante_goleador(new_df)

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
652,Elkeson,Brazil,1989-07-13,Grêmio Foot-Ball Porto Alegrense,Forward (association football),1.80,9,59,Campeonato Brasileiro Série A
1261,Mersudin Ahmetović,Bosnia & Herzegovina,1985-03-19,FK Sarajevo,Forward (association football),1.91,9,54,Premier League of Bosnia and Herzegovina
2119,Nahikari García,Spain,1997-03-10,Real Madrid Femenino,Forward (association football),1.59,9,104,Primera División (women)
2249,Jeison Medina,Colombia,1995-02-27,Al-Shamal SC,Forward (association football),1.80,9,65,Qatar Stars League
2423,Abdulfattah Adam,Saudi Arabia,1995-01-01,Abha Club,Forward (association football),1.80,9,51,Saudi Professional League
2814,Aleksey Khodnevich,Belarus,1987-11-02,FC Ostrovets,Forward (association football),1.94,9,67,Belarusian First League


### Regra 5

Acha todos os jogadores concorrentes, ou seja, que jogam no mesmo clube e jogam na mesma posição

In [51]:
def jogadores_concorrentes(df):
    return [(j1, j2, pos, clube) for j1, pos, clube in zip(df['nome'], df['posicaoLabel'], df['clubeNome'])
                                    for j2, pos2, clube2 in zip(df['nome'], df['posicaoLabel'], df['clubeNome'])
                                    if j1 < j2 and pos == pos2 and clube == clube2]

In [52]:
jogadores_concorrentes(new_df)

[('Nen Sothearoth',
  'Nub Tola',
  'Defender (association football)',
  'Nagaworld FC'),
 ('Seon Power',
  'Sor Rotana',
  'Defender (association football)',
  'Prey Veng FC'),
 ('Sa Ty', 'Wut Tola', 'Forward (association football)', 'Visakha FC'),
 ('Reung Bunheing', 'Sa Ty', 'Forward (association football)', 'Visakha FC'),
 ('Reung Bunheing',
  'Wut Tola',
  'Forward (association football)',
  'Visakha FC'),
 ('Paulo Victor (footballer, born 1994)',
  'Sa Ty',
  'Forward (association football)',
  'Visakha FC'),
 ('Paulo Victor (footballer, born 1994)',
  'Reung Bunheing',
  'Forward (association football)',
  'Visakha FC'),
 ('Paulo Victor (footballer, born 1994)',
  'Wut Tola',
  'Forward (association football)',
  'Visakha FC'),
 ('Lee Jae-gun', 'Sa Ty', 'Forward (association football)', 'Visakha FC'),
 ('Lee Jae-gun',
  'Reung Bunheing',
  'Forward (association football)',
  'Visakha FC'),
 ('Lee Jae-gun',
  'Paulo Victor (footballer, born 1994)',
  'Forward (association footbal

### Regra 6

Acha todos os jogadores rivais, ou seja, que jogam na mesma liga e em clubes diferentes

In [53]:
def jogadores_rivais(df):
    parceiros = jogadores_parceiros(df)
    return [(j1, j2, liga) for j1, clube1, liga in zip(df['nome'], df['clubeNome'], df['liga'])
                              for j2, clube2, liga2 in zip(df['nome'], df['clubeNome'], df['liga'])
                              if j1 < j2 and liga == liga2 and clube1 != clube2
                              and (j1, j2) not in parceiros and (j2, j1) not in parceiros]

In [54]:
jogadores_rivais(new_df)

[('Nub Tola', 'Robert Lopez Mendy', 'Cambodian Premier League'),
 ('Nub Tola', 'Sa Ty', 'Cambodian Premier League'),
 ('Nub Tola', 'Reung Bunheing', 'Cambodian Premier League'),
 ('Nub Tola',
  'Paulo Victor (footballer, born 1994)',
  'Cambodian Premier League'),
 ('Nub Tola', 'Valeriy Hryshyn', 'Cambodian Premier League'),
 ('Nub Tola', 'Takumu Nishihara', 'Cambodian Premier League'),
 ('Nub Tola', 'Wut Tola', 'Cambodian Premier League'),
 ('Nub Tola', 'Phuong Soksana', 'Cambodian Premier League'),
 ('Nub Tola', 'Thierry Bin', 'Cambodian Premier League'),
 ('Nub Tola',
  'Pak Song-chol (footballer, born 1987)',
  'Cambodian Premier League'),
 ('Nub Tola', 'Takashi Odawara', 'Cambodian Premier League'),
 ('Nub Tola', 'Víctor Blasco', 'Cambodian Premier League'),
 ('Nen Sothearoth', 'Robert Lopez Mendy', 'Cambodian Premier League'),
 ('Nen Sothearoth', 'Sa Ty', 'Cambodian Premier League'),
 ('Nen Sothearoth', 'Reung Bunheing', 'Cambodian Premier League'),
 ('Nen Sothearoth',
  'Paulo V

### Regra 7

Acha todos os goleiros bons, ou seja, jogadores que jogam na posição de goleiro, que tenham pelo menos 1.90 de altura e que tenham feito pelo menos 1 gol

In [55]:
def goleiro_bom(df):
    return df[(df['posicaoLabel'] == "Goalkeeper (association football)") &
              (df['altura'] >= 1.90) &
              (df['allGols'] >= 1)]

In [56]:
goleiro_bom(new_df)

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
90,"Vinícius (footballer, born 1984)",Brazil,1984-11-09,Clube do Remo,Goalkeeper (association football),1.90,1,1,Campeonato Brasileiro Série C
237,"Everson (footballer, born 1990)",Brazil,1990-07-22,Clube Atlético Mineiro,Goalkeeper (association football),1.92,22,1,Campeonato Mineiro
240,"Everson (footballer, born 1990)",Brazil,1990-07-22,Clube Atlético Mineiro,Goalkeeper (association football),1.92,22,1,Campeonato Brasileiro Série A
455,Pegorari,Brazil,1991-07-12,Esporte Clube Juventude,Goalkeeper (association football),1.91,1,1,Campeonato Brasileiro Série A
775,Eduardo Martini,Brazil,1979-01-11,Grêmio Esportivo Brasil,Goalkeeper (association football),1.92,1,1,Campeonato Brasileiro Série D
1400,"Paulo Vítor (footballer, born 1988)",Brazil,1988-11-21,G.D. Chaves,Goalkeeper (association football),1.90,28,1,Primeira Liga
2358,Vladimir Stojković,Serbia,1983-07-28,Al-Fayha FC,Goalkeeper (association football),1.95,88,1,Saudi Professional League
2835,Vyacheslav Kaminsky,Belarus,1988-07-05,FC Ostrovets,Goalkeeper (association football),1.92,1,1,Belarusian First League
2914,Igor Dovgyallo,Belarus,1985-07-15,FC Naftan Novopolotsk,Goalkeeper (association football),1.90,1,1,Belarusian First League


### Regra 8

Acha todos os jogadores camisa 10 da shoppe, ou seja, jogadores que tem o número de camisa 10, e que não tenham feito nenhum gol

In [57]:
def jogadores_camisa_10_da_shoppe(df):
    return df[(df['numeroCamisa'] == 10) & (df['allGols'] == 0)]

In [58]:
jogadores_camisa_10_da_shoppe(new_df)

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
1193,Michael Modubi,South Africa,1985-04-22,K.F.C. Dessel Sport,Midfielder,1.76,10,0,Belgian National Division 1
2070,Ismael Gutiérrez,Spain,2000-10-10,CF La Nucía,Midfielder,1.76,10,0,Primera División RFEF
2235,Salaah Al-Yahyaei,Oman,1998-08-17,Qatar SC,Midfielder (association football),1.80,10,0,Qatar Stars League
2254,Jameel Al-Yahmadi,Oman,1996-07-27,Al-Markhiya SC,Forward (association football),1.68,10,0,Qatar Stars League
2290,Jameel Al-Yahmadi,Oman,1996-07-27,Al-Markhiya SC,Midfielder (association football),1.68,10,0,Qatar Stars League
2550,Thaar Al-Otaibi,Saudi Arabia,1999-08-14,Al-Hazem F.C.,Midfielder,1.73,10,0,Saudi First Division League
